# Training a classifier on all datasets and establishing a regularization parameter

In [1]:
import os
import sys
import pickle
import numpy as np
import xarray as xr
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
sys.path.append('../')

import core.core_functions as cf
import core.dataset_functions as df
import core.plot_functions as pf

land_masked = True
global_mean = True
# set to a not-None value to draw the same random samples for repeated calls
random_init = 6546

## Load and prepare training samples

In [ ]:
# NOTE: we want the same amount of samples in the observation and the model category
# therefore we call get_samples separately for both categories and change the number
# of time steps selected
samples_model = cf.get_samples(
    period=slice('1982', '2001'), 
    land_masked=land_masked,
    global_mean=global_mean,
    time_steps=200, 
    random_init=random_init, 
    verbose=True, 
    datasets=df.model_names,  # 43 models
)

samples_obs = cf.get_samples(
    period=slice('1982', '2001'), 
    land_masked=land_masked,
    global_mean=global_mean,
    time_steps=2150,  # 200*43/4
    random_init=random_init, 
    verbose=True,
    datasets=df.observation_names,  # 4 observations
)

samples = xr.concat([samples_model, samples_obs], dim='sample')

In [ ]:
XX = samples.values
yy = df.get_category_ids(samples['dataset_name'].values)

nan_mask = np.any(np.isnan(XX), axis=0)
XX = XX[:, ~nan_mask]

print('Number of features:', XX.shape[1])
print('Number of samples per category:', ', '.join(np.unique(yy, return_counts=True)[1]))

## Set up cross validation to establish regularization

In [ ]:
logreg = LogisticRegression(penalty='l2', solver='liblinear')
grid = {'C': np.logspace(-5, -1, 20)}

# 5-fold cross validation on shuffled training data
XX, yy = shuffle(XX, yy, random_state=random_init)
cv = 5

# alternatively we could do folds for each dataset of dataset group
# this is very slow though
# groups = samples['dataset_name'].values  # one fold for each dataset
# groups = df.get_groups(samples['dataset_name'].values)  # one fold for each group
# cv = GroupKFold(n_splits=len(np.unique(groups)))
# print('Number of dataset groups (=folds):', cv.n_splits)
# cv.split(XX, yy, groups)

logreg_cv = GridSearchCV(
    estimator=logreg, 
    param_grid=grid, 
    cv=cv, 
    n_jobs=20, 
    return_train_score=True,
)

## Train classifier

In [ ]:
logreg_cv.fit(XX, yy)
print(f'{logreg_cv.best_params_=}')

## Save trained classifier

In [ ]:
savename =  'logreg{}.sav'.format(('_lm' if land_masked else '') + ('_gm' if global_mean else ''))
pickle.dump(
    logreg_cv.best_estimator_, 
    open(os.path.join('../../data/trained_classifiers', savename), 'wb'))

## Training accuracy

In [ ]:
logreg_cv.score(XX, yy)

## Plot classifier properties

### Regularization strength

In [ ]:
pf.plot_hyper_param(logreg_cv, 'C', xscale='log')

### Regression weights

In [ ]:
pf.plot_coef_map(logreg_cv.best_estimator_)

## Reliability diagram

In [ ]:
pf.plot_reliability_diagram(logreg_cv, XX, yy)